Linking without duplication


Splink steps:

1) Prepare data
2) Exploratory analysis 
3) Blocking:
    - Create your blocking rules for prediction
4) Estimate model parameters:
    - Define comparisons
    - Define your model using the link type, the comparisons and your blocking rules
    - Estimate the parameters of the model and visualise to see what the model is doing
5) Predict results
    - Generate match_weight and match_probability scores
    - Assign records to clusters 
6) Visualise predictions to see what the model is doing
7) Evaluate against labelled data (if poss!)

Data prerequisites

Unique IDs:
- Each input dataset must have a unique ID column (unique within the dataset)
- By default, Splink assumes this will be called unique_id, but this can be changed

Conformant datasets:
- If using multiple datasets, they must share the same column names and data formats (order doesn't matter)

Cleaning:
- Ensure consistency by cleaning the data, e.g. standardising date formats, matching text case, handling invalid data
- Usual data cleaning of obvious errors

Ensure nulls are consistently and correctly represented:
- Make sure that nulls are represented as true nulls, not empty strings - splink handles these types of value differently

In [ ]:
#import packages

import splink #https://moj-analytical-services.github.io/splink/

from splink.internals.duckdb.database_api import DuckDBAPI
#splink implements data linking computations by generating SQL and submitting the SQL statements to a backend of our choice
#syntax is almost exactly the same between backends 
#worth using DuckDB first to explore data as it is fast, then migrate to a different backend if desired

from splink import block_on
from splink import Linker, SettingsCreator

from splink.datasets import splink_datasets #some datasets available with splink for practice https://moj-analytical-services.github.io/splink/api_docs/datasets.html
from splink.exploratory import completeness_chart #https://moj-analytical-services.github.io/splink/api_docs/exploratory.html
from splink.exploratory import profile_columns


from splink.blocking_analysis import cumulative_comparisons_to_be_scored_from_blocking_rules_chart #https://moj-analytical-services.github.io/splink/api_docs/blocking_analysis.html
from splink.blocking_analysis import count_comparisons_from_blocking_rule

import splink.comparison_library as cl #https://moj-analytical-services.github.io/splink/api_docs/comparison_library.html

Data prerequisites

Unique IDs:
- Each input dataset must have a unique ID column (unique within the dataset)
- By default, Splink assumes this will be called unique_id, but this can be changed

Conformant datasets:
- If using multiple datasets, they must share the same column names and data formats (order doesn't matter)

Cleaning:
- Ensure consistency by cleaning the data, e.g. standardising date formats, matching text case, handling invalid data
- Usual data cleaning of obvious errors

Ensure nulls are consistently and correctly represented:
- Make sure that nulls are represented as true nulls, not empty strings - splink handles these types of value differently

In [ ]:
#import data
df = splink_datasets.fake_1000
df = df.drop(columns = ["cluster"])
#split dataset into two
df1 = df.sample(frac = 0.5)
df2 = df.drop(df1.index)

Exploratory analysis

- Splink has a range of visuals to help with EDA:

In [ ]:
df1.head(10)

In [ ]:
df2.head(10)

Analyse nulls - columns with higher numbers of nulls are less useful for data linking

In [ ]:
completeness_chart(
    [df1, df2],
    cols = ["first_name", "surname", "dob", "city", "email"],
    db_api=DuckDBAPI(),
    table_names_for_chart=["df_left", "df_right"],
)

- The completeness chart shows us the % of nulls in the dataset

Assess distribution of values
- Columns with higher cardinality (number of distinct values) are more useful for linking
- Columns that are more equally distributed are more useful for linking

In [ ]:
#profile_columns
#dist plot showing the count of values at each percentile
#top n chart showing the counc of the top n values within column
#bottom n chart showing the count of the bottom n values in the column
profile_columns([df1, df2], db_api = DuckDBAPI(), top_n = 10, bottom_n = 5)


Blocking

Choosing blocking rules to optimise runtime
- To link records, we need to compare pairs of records and decide which pairs are matches
- For most large datasets, it won't be computationally possible to compare every row with every other row (the number of comparisons rises quadratically with the number of records)
- To decide which ones to compare we use blocking rules - these specify which pairwise comparisons to generate
- These are defined as SQL expressions, e.g.:

    from splink import block_on
    block_on("first_name", "surname")

    =
    
    SELECT *
    FROM input_tables as l
    INNER JOIN input_tables as r
    on l.first_name = r.first_name AND l.surname = r.surname


The aim of blocking rules are:
- Eliminate enough non-matching comparison pairs so the we can computationally handle the number of pairwise comparisons
- Eliminate as few true matching pairs as possible
- Splink has some tools to help us choose effective rules
- Lots of strict blocking rules are usually better than few loose rules; individually strict blocking rules are likely to exclude lots of true matches, multiple strict rules will make it implausible that a truly matching record gets missed, e.g.:
    block_on("first_name", "dob")
    will retain all matching pairs except those with errors or nulls in the first name or dob fields
    and
    block_on("email")
    will retain all matching pairs except those with errors or nulls in the email column
- Individually we would probably miss true matches where the records contain typos but between them, it's unlikely that the same records would have typos in both fields
- If we add more strict blocking rules it becomes less likely that a record would get through the cracks here
- Choosing good blocking rules is important to building a good model

In [ ]:
#counting comparisons created by a single rule
#this is a good idea so we know that we are not generating too many records and wasting time computing them

br = block_on("first_name") #inital of first name and surname match
counts = count_comparisons_from_blocking_rule(
    table_or_tables=[df1, df2],
    blocking_rule=br,
    link_type="link_only",
    db_api= DuckDBAPI()
)
counts

In [ ]:
#counting comparisons created by a single rule
#this is a good idea so we know that we are not generating too many records and wasting time computing them

br = block_on("surname") #inital of first name and surname match
counts = count_comparisons_from_blocking_rule(
    table_or_tables=[df1, df2],
    blocking_rule=br,
    link_type="link_only",
    db_api= DuckDBAPI()
)
counts

In [ ]:
#counting the number of comparisons created by a list of blocking rules
blocking_rules_for_analysis = [
        block_on("first_name"),
        block_on("surname")
]

#cumulative_comparisons_to_be_scored_from_blocking_rules_chart(table_or_tbles, blocking_rules, link_type, db_api, unique_id_column_name = "unique_id", 
#max_rows_limit = int(1000000000.0), source_dataset_column_name = None)
cumulative_comparisons_to_be_scored_from_blocking_rules_chart(
    table_or_tables=[df1, df2],
    blocking_rules=blocking_rules_for_analysis,
    db_api = DuckDBAPI(),
    link_type="link_only"
)

Building and estimating the model
- Estimating the model will help us understand the relative importance of different parts of the data for data linking
- The relative importance is captured in the partial match weights (which are added to compute the overall match score)
- To build a model, we define the partial match weights that splink is to estimate by defining how the data in the records should be compared
- Comparisons represent how data from one or more input columns is compared
- A model is composed of many Comparisons, which between them assess the similarity of all the columns being used for linking the data
- Each comparison contains two or more ComparisonLevels which define n discrete graduations of similarity between input columns within the Comparison
- Splink has a library of comparison functions which are split into:
    - generic comparison functions which apply a particular fuzzy matching pattern (e.g. levenshtein distance)
    - tailored comparison functions for specific data types

- There are 3 ways of specifying comparisons:
    - Using "out-of-the-box" Comparisons
    - Composing pre-defined ComparisonLevels
    - Writing a full dictionary spec of a Comparison by hand

- "Out-of-the-box" Comparisons:
    - The ComparisonLibrary has pre-baked similarity functions that cover many common use cases
    - These functions generate an entire Comparison, composed of ComparisonLevels
    - Include non-data-specific and data-specific comparisons

- Composing pre-defined ComparisonLevels
    - Compose our own Comparisons

- Full dictionary spec
    - All Comparisons are eventually turned into a dictionary
    - The library functions are convenience functions that provide a shorthand way to produce valid dictionaries, but we can specify our own Comparisons directly as a dictionary to get maximum control

In [10]:
settings = SettingsCreator(
    link_type="link_only",
    blocking_rules_to_generate_predictions=[
        block_on("first_name"),
        block_on("surname")
    ],
    comparisons=[
        cl.NameComparison("first_name"),
        cl.NameComparison("surname"),
        cl.DateOfBirthComparison("dob",
            input_is_string = True,
            invalid_dates_as_null = True
        ),
        cl.ExactMatch("city").configure(term_frequency_adjustments=True),
        cl.EmailComparison("email"),
    ]
)

linker = Linker(
    [df1, df2],
    settings,
    db_api = DuckDBAPI(),
    input_table_aliases=["df_left", "df_right"],
)

Term frquency adjustments
- The Fellegi-Sunter model doesn't account for skew in the distributions of linking variables
- Consider, for example, a binary gender variable were males outnumber females by 10:1
- This doesn't affect the m probability - given that two records are a match, the gender fields should match with roughly the same probability for males and females
- The u probability, however is affected - given that two records are not a match, it is much more likely that both records will be male than that they will both be female - u probability is too low for the more common value and too high otherwise
- One option might be to create different comparison levels for the gender variable, but this means we have to calculate more probabilities, and we would need many comparison levels if we had higher cardinality values
- To deal with the problem we can add an independent TF adjustment term for each comparison

Estimating lambda
- 𝜆 = Pr(Records match) = probability that two records match
- In some cases we might know lambda, for example, if we know that there is a one-to-one match between datasets
- In most cases we don't know this, so we combine a set of deterministic matching rules and a guess of the recall corresponding to these rules

In [ ]:
deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    block_on("email"),
]

linker.training.estimate_probability_two_random_records_match(deterministic_rules, recall = 0.7)

Estimating m and u probabilities

- m and u probabilities quantify the strength of the evidence we have in our data 
- m = Pr(scenario|records match)
- u - Pr(scenario|records do not match)
- What is important is the relative size of these values - this is the Bayes Factor:
        K = m/u = Pr(scenario|records match)/Pr(scenario|records do not match)
- Bayes Factors act as a relative multiplier that increases or decreases the overall prediction of whether the records match

Estimating u probabilities
- Once we have lambda, we can estimate u probabilities
- We use the estimate_u_using_random_sampling method which samples random pairs of records, since most random pairs will be non-matches
- Over these non-matches we compute the distribution of ComparisonLevels for each Comparison

In [ ]:
linker.training.estimate_u_using_random_sampling(max_pairs=1e8, seed = 1)

- We now need to estimate m - for this we have to have some idea of what the true matches are
- We can use an iterative maximum likelihood approach called Expectation Maximisation:
    - Iterative optimisation method that finds maximum likelihood of parameters in models that have unobserved latent variables (unobserved variables in models that can only be inferred indirectly through their effects on observed variables)
    - Made up of an estimation (E) step and maximisation (M) step:
        - E- compute the latent variables (expectation of the log-likelihood (log of likelihood function, which measures the goodness of fir between data nd the model) using the current parameter estimates)
        - M- determine the parameters that maixmise the expected log-likelihood obtained in E step, and update the model paremeters based on the estimated latent variables
- This estimates the m values by generating pairwise record comparisons and using them to maximise a likelihood function
- Each estimation pass requires us to configure an estimation blocking rule to reduce the number of record comparisons so it is manageable

In [ ]:
session_dob = linker.training.estimate_parameters_using_expectation_maximisation(block_on("dob"))
session_email = linker.training.estimate_parameters_using_expectation_maximisation(block_on("email"))
session_first_name = linker.training.estimate_parameters_using_expectation_maximisation(block_on("first_name"))

In [ ]:
#now we can visualise the model parameters
#see the final estimated match weights
linker.visualisations.match_weights_chart()

- The match weights chart shows results of a trained Splink model
- Each comparison is represented in a bar chart - eahc bar shows evidence for two records being a match for each comparison level
- The first bar is our prior - bayesian prior - represents our belief that two random records will be a match

Things to focus on:

Match weights should gradually reduce within a comparison:
- Comparison levels are order-dependent - so the most similar levels come first and the levels get gradually less similar
- So we would expect that the match weight will reduce as we move down the levels

We might want to combine comparison levels that are very similar
- Comparisons are broken up into levels to show different levels of similarity
- Because of this, we expect the amount of evidence (match weight) to vary between comparison levels
- Two levels with the same match weight do not provide the model with any additional information, so we should combine very similar levels into a similar level

We might want to simplify a model that has a number of highly predictive features
- Where we have a large variation between comparison levels, it indicates that we have a highly predictive feature (consider the difference between and exact match on email and all other comparisons on email)
- If we have a lot of highly predictive features, we might consider simplifying the model using the more predictive features

Logically walk through each comparison level
- Check the amount of evidence (match weight) that has been allocated by the model for each comparison level
- Logically consider each level and how much evidence matches give us - do they make sense knowing our data

In [ ]:
#m and u values
linker.visualisations.m_u_parameters_chart()

- Left shows estimated m probabilities - the probability of a given comparison level when two records are a match - the proportion of matching records allocated to the comparison level
- Right shows estimated u probabilities - the probability of a given comparison level when two records do not match - the proportion of non-matching records allocated to the comparison level

Things to focus on:

Logically walk through each comparison level
- Consider, for example, how often exact matches and fuzzy levels occurence in non-matching comparisons
- Consider the cardinality of the features - high cardinality features will generally have lower likelihood of "all other comparisons"

In [ ]:
#comparisons
linker.visualisations.parameter_estimate_comparisons_chart()

- Shows how parameter estimates have differed across different estimation methods

Predicting match weights
- Linker.predict() runs the model
- This generates all pariwise record comparisons that match at least one of the blocking_rules_to_generate_predictions
- Uses the rules we defined in the comparisons to evaluate the similarity of the input data
- Uses the estimated match weights, applying term frequency adjustments where we have set this as true, to produce the final_match_weight and match_probability scores
- We can also define a threshold_match_probability or theshold_match_Weight to drop any rows where the predicted score is below a certain threshold

Bayes Factors -> probabilities

- The prior is our existing belief that two random records match (our belief of the scenario before we have any evidence)
- The posterior is our belief that two records match given the evidence we have (the data we have about the records)

- Mathematically:

    posterior odds = prior odds * Bayes Factor

- And Bayes Theorem is:

    Pr(a|b) = Pr(b|a) * Pr(a) / Pr(b)

- or:

    posterior probability = likelihood * prior probability / evidence

- so if we consider one column (e.g. first name):

    Pr(match|first name matches) = Pr(first name matches|match) * Pr(match) / Pr(first name matches)

- which we can also write as:

    Pr(match|first name matches) = Pr(first name matches|match) * Pr(match) / Pr(first name matches|match) * Pr(match) + Pr(first name matches|non match) * Pr(non match)

- m = Pr(scenario|records match)
- u - Pr(scenario|records do not match)

- so this is the same as:

    posterior probability = m * prior probability / m * prior probability + u * (1 - prior probability)

- Odds is:

    odds = p / 1-p

- so:

    posterior odds = prior / 1 - prior  m / u

- so for a specific scenario:

    posterior odds = prior odds * Bayes Factor

- This formula can account for data in multiple scenarios (e.g. a match in on multiple parameters, or a match on some and not on others) (Naive Bayes classifier):

    posterior odds = prior odds * Bayes Factor 1 * Bayes Factor 2 ... * Bayes Factor n

- Which means:
    posterior odds = prior odds * m1 m2 ... mn / u1 u2 ... un



In [ ]:
results = linker.inference.predict(threshold_match_probability = 0.9)

In [ ]:
results.as_pandas_dataframe(limit = 5)

Clustering records
- From the linker.predict we get a list of pairwise record comparisons and their scores
- We now need to convert the pairwise results into clusters

In [ ]:
clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(
    results, threshold_match_probability=0.5
)
clusters.as_pandas_dataframe(limit = 10)

In [ ]:
sql = f"""
select *
from {results.physical_name}
limit 2
"""

linker.misc.query_sql(sql)